# Clustering

aca anotar lo q se quiere probar

recordar utilizar metricas de comparación

https://huggingface.co/julien-c/distilbert-feature-extraction

CountVectorizer, TFIDF, LSA, LDA, SBERT, spaCy

ideas:
    (1,2) ngrams of Qpos + Apos   
    (1,2) ngrams of Qlemmas
    (1,2) ngrams of Alemmas
    (1,2) ngrams of AQAQlemmas


ya tengo el codigo para agregar features


In [13]:
from TCA import *

import pandas as pd
import numpy as np

df = pd.read_csv("db/features.csv", index_col=0)
df = df.set_index("id")

test_df = pd.read_csv("db/test_db.csv", index_col=0)

n_clusters = 30

## Vectorial Representations

### CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold

def cv_ngrams(docs, range=(2,3), min_df=20):
    
    #2 y 3 ngrams, minimo 20 (no se como elegir le numero correcto)
    cv = CountVectorizer(min_df=min_df, ngram_range=range)

    
    #Create, Normalize and Reduce 
    vspace = cv.fit_transform(docs)
    vspace = normalize(vspace, axis=1, norm="max")
    vspace = VarianceThreshold(threshold=1e-3).fit_transform(vspace)

    return vspace

vs = cv_ngrams(df["Q"])
vs


<98182x704 sparse matrix of type '<class 'numpy.float64'>'
	with 338111 stored elements in Compressed Sparse Row format>

### Embeddings


In [31]:
from sklearn.cluster import KMeans

kmc = KMeans(n_clusters=n_clusters, random_state=0)
kmc.fit(X=vs)
df["label1"] = kmc.labels_

In [29]:

#vemos los primeros clusters, al ser con ngrams  la forma en la que comienza la pregunta tiene mucho peso
def print_kmc_clus(n=5):
    import random
    q_clus = [[] for i in range(n_clusters)]

    for sentence_id, cluster_id in enumerate(kmc.labels_):
        q_clus[cluster_id].append(df.iloc[sentence_id].Q)

    for i, cluster in enumerate(q_clus):
        print("Cluster ", i+1)
        for tw in random.sample(cluster, n):
            print(tw)

Cluster  1
Who is the worst kind of Reddit user?
Who is the trippiest person alive today?
Who is the best WWE wrestler of all time?
Who is the forbidden fruit in your life?
Who is the coolest person in history?
Cluster  2
It looks could kill, what’s your body count?
What's Your Personal Heaven?
What's your hypnosis story?
What’s your best or worst failed romance plan?
What’s your favourite ice cream flavour?
Cluster  3
What is the greatest comeback you've ever heard?
What is the dumbest thing you've ever done?
What is the wisest thing you've ever been told?
What is the scariest movie you've ever seen?
What is the best catch you've ever made?
Cluster  4
What is the worst gift you have ever given?
What is the best product you have ever purchased?
what is the spiciest food you have ever eaten?
What is the first game you have ever played?
What is the grossest thing you have ever seen?
Cluster  5
What is your prediction for 2022?
What is your weirdest funeral experience?
What is your perfec

TypeError: Cannot index by location index with a non-integer key

In [ ]:
#continuo viendo que funciona

#embeddings preentrenados
import gensim.downloader

#word2vec-google-news-300
w2v = gensim.downloader.load('glove-twitter-25')


In [ ]:
w2v["mauro"]

array([-0.657   ,  0.27942 ,  0.6727  , -0.1725  , -0.91333 , -0.96729 ,
       -1.7812  ,  1.244   , -0.029866, -1.1127  ,  0.47211 , -0.87454 ,
       -0.92561 , -1.11    ,  1.1491  , -0.35654 , -0.60955 ,  0.20226 ,
       -0.13373 ,  0.56002 ,  0.25254 ,  0.57305 ,  0.14504 , -0.21456 ,
       -0.29104 ], dtype=float32)

In [53]:
df["oul46s"]

KeyError: 'oul46s'

Q                          First time drinking, what do I do?
Qscore                                                      7
Qless                                First time drinking, I ?
Qlemmas                             First time drinking , I ?
Qpos        [('First', 'JJ'), ('time', 'NN'), ('drinking',...
Qtoks            ['First', 'time', 'drinking', ',', 'I', '?']
ANS         try not mix drinks.., if you start with spirit...
ANSscore                                                 12.0
Aless        try mix drinks.., start spirits finish spirits..
Alemmas      try mix drink .. , start spirit finish spirit ..
Apos        [('try', 'VB'), ('mix', 'JJ'), ('drinks', 'NNS...
Atoks       ['try', 'mix', 'drinks', '..', ',', 'start', '...
label1                                                     13
Name: 100b8y, dtype: object

In [41]:


int(df[df.id == test_df.iloc[1][0]].label1) ==  int(df[df.id == test_df.iloc[1][1]].label1)

False

In [52]:
def get_measure():
    n = len(test_df)
    count = 0
    for i in range(n):
        if int(df[test_df.iloc[i][0]].label1) ==  int(df[test_df.iloc[i][1]].label1):
            count += 1
        
    return count
get_measure()
   

KeyError: 'oul46s'